# <span style="color:#91299A">Основа аспирантской диссертации - проблема наблюдаемости системы</span> 

![](../storage/banners/7.jpg)

### <span style="color:#0ab49a">Согласно статье</span> <span style="color:#A254FC">038 (Shauying R.K.) Observability of Nonlinear Systems</span> 

#### <span style="color:#2c3e50">Начало</span> 

In [1]:
from sympy import *
import numpy as np

# Общие параметры
h_orb = 350e3
RadiusEarth = 6371000.
Radius_orbit = RadiusEarth + h_orb
mu = 398576057600000.06
ω_orb = np.sqrt(mu / (Radius_orbit ** 3))
v_orb = ω_orb * Radius_orbit

# Расчёт плотности атмосферы -> https://www.grc.nasa.gov/www/k-12/airplane/atmosmet.html
t = -131.21 + 0.00299 * h_orb
p = 2.488 * ((t + 273.1) / 216.6) ** -11.388
ρ = p / (0.2869 * (t + 273.1))

print(f"Высота орбиты: {int(h_orb // 1e3)} км\nПериод орбиты: {round((2*np.pi/ω_orb) / 3600, 2)} часов\nПлотность атмосферы: {ρ} кг/м³")

Высота орбиты: 350 км
Период орбиты: 1.52 часов
Плотность атмосферы: 2.7797537885625094e-11 кг/м³


#### <span style="color:#2c3e50">Алгоритм</span>

<span style="color:#2b817d">Примечание:</span>     $km \geq n$

In [43]:
# Функции вспомогательные
def quat2matrix(L) -> Matrix:
    """Преобразует единичный кватернион в матрицу поворота
    :param q: Кватернион, list длинны 4
    :return: Матрица поворота, sympy.Matrix"""
    w, x, y, z = L
    A = Matrix(np.eye(3))
    A[0, 0] = 1 - 2 * y ** 2 - 2 * z ** 2
    A[0, 1] = 2 * x * y + 2 * z * w
    A[0, 2] = 2 * x * z - 2 * y * w
    A[1, 0] = 2 * x * y - 2 * z * w
    A[1, 1] = 1 - 2 * x ** 2 - 2 * z ** 2
    A[1, 2] = 2 * y * z + 2 * x * w
    A[2, 0] = 2 * x * z + 2 * y * w
    A[2, 1] = 2 * y * z - 2 * x * w
    A[2, 2] = 1 - 2 * x ** 2 - 2 * y ** 2
    return A

def local_dipol(r: list, r_abs, ind: str, q: list, distortion=0):
    """Возвращает усиление антенны полуволногого диполя
    :param r: направление на аппарат в орбитальной системе координат (ОСК)
    :param r_abs: модуль вектора r (для таких лентяев как йа)
    :param ind: ось x/y/z, по которой направлена антенна в собственной системе координат (ССК)
    :param q: кватернион поворота ОСК -> ССК
    :param distortion: искажение диаграммы направленности (несиммеричность)"""
    r_ = quat2matrix(q) @ Matrix(r)
    cos_a = (r_[0]*int(ind=='x') + r_[1]*int(ind=='y') + r_[2]*int(ind=='z'))/r_abs
    sin_a = (sqrt(r_[1]**2 + r_[2]**2)*int(ind=='x') + \
             sqrt(r_[0]**2 + r_[2]**2)*int(ind=='y') + \
             sqrt(r_[0]**2 + r_[1]**2)*int(ind=='z'))/r_abs
    aside = ((r[1]+r[2])*int(ind=='x') + \
             (r[0]+r[2])*int(ind=='y') + \
             r[2]*int(ind=='z'))/r_abs
    return cos(cos_a * pi / 2) / sin_a + distortion * cos_a**2 + distortion * aside

def get_vars(name: str, n: int):
    """Генерит символьные переменные с одинаковым названием и индексами 0...(n-1)
    :param name: Название переменных (без учёта индекса)
    :param n: Количество переменных"""
    s = ""
    for i in range(n):
        s += f"{name}_{i} "
    return var(s, real=True)

def get_func(name: str, n: int):
    """Генерит символьные функции с одинаковым названием и индексами 0...(n-1)
    :param name: Название функций (без учёта индекса)
    :param n: Количество функций"""
    return [Function(f"{name}_{i}", real=True)(t) for i in range(n)]

def get_params(n: int):
    """Экспресс-инициализация переменных и функций для исследования критериев наблюдаемости
    :param n: Количество чипсатов, кинематические параметры которых нужно найти/оценить"""
    t, ω = var("t ω", real=True)
    r_cube = get_func(f"r^c", 3)
    v_cube = get_func(f"v^c", 3)
    x = get_func(f"x", n)
    y = get_func(f"y", n)
    z = get_func(f"z", n)
    vx = get_func(f"v^x", n)
    vy = get_func(f"v^y", n)
    vz = get_func(f"v^z", n)
    wx = get_func(f"w^x", n)
    wy = get_func(f"w^y", n)
    wz = get_func(f"w^z", n)
    q0 = get_func(f"q^0", n)
    qx = get_func(f"q^x", n)
    qy = get_func(f"q^y", n)
    qz = get_func(f"q^z", n)
    return t, ω, r_cube, v_cube, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz

def save_reports(report_list: list, filename: str) -> None:
    """Сохраняет в компактном виде несколько выводов в один
    :param report_list: Строки вывода, запрашиваемые к объединению
    :param filename: Название текстовика, в котором будет храниться информация"""
    common_report = report_list[0]
    for i in range(len(report_list) - 1):
        common_report += ("\n" + "-" * 100)*2 + "\n" + report_list[i + 1]
    f = open("storage/observability_reoprt_" + filename + ".txt", "w")
    f.write(common_report)
    f.close()

def read_reports(filename: str) -> str:
    """Считывает выводы, сохранённые в один файл
    :param filename: Название текстовика, в котором хранится информация"""
    f = open("storage/observability_reoprt_" + filename + ".txt", "r")
    common_report = f.read()
    f.close()
    return common_report

def print_spectrm_rank(J_numb):
    for tol in [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10, 1e-11, 1e-12, 1e-13, 1e-14, 1e-15, 1e-20]:
        print(f"Ранг матрицы: {np.linalg.matrix_rank(J_numb, tol=tol)} (tol={tol})")

In [75]:
def ShauyingObservabilitySufficientCondition(n: int, X: list, Y: list, my_diff, testprint: bool = False, hand_written_deriv: int = None):
    """Проверка достаточного условия наблюдаемости системы. Проверка равномерного отношения миноров матрицы наблюдаемости.
    :param n: Количество чипсатов
    :param X: Список неизвестных параметров, которые необходимо найти
    :param Y: Список известных параметров (измерений системы в t₀=0)
    :param my_diff: Функция взятия производной по времени
    :param testprint: Флаг вывода экстра-информации"""
    report = f"\033[1mКоличество чипсатов: {n}\033[0m\n"
    print(report)

    # Количество одномоментных измерений
    l = len(Y)
    # Требуемое количество существующих производных функции измерения
    k = int(len(X) // len(Y)) if hand_written_deriv is None else hand_written_deriv
    txt = f"" if hand_written_deriv is None else f"\033[1mВнимание! Рассчитывается не отношение миноров, а ранг расширенного Якобиана\033[0m\n"
    txt += f"Неизвестные: n = {len(X)} (на каждый чипсат по {int(len(X) // n)} параметров)\nИзвестные: l = {l}\n∃ производные порядка k = {len(X) / len(Y)} (Должна быть целой!)"
    report += txt + "\n"
    print(txt)
    
    # Матрица наблюдаемости системы
    H = Matrix([[Y[0] for ll in range(l)] for kk in range(k)])
    H_one_line = []
    for kk in range(k):
        for ll in range(l):
            tmp = Y[ll] if kk == 0 else my_diff(H[kk - 1, ll])
            if testprint:
                print(f"_расчёт матрицы H_: k={(kk+1)}/{k}, l={(ll+1)}/{l}")
            H[kk, ll] = tmp
            H_one_line += [tmp]
    H = Matrix(H_one_line)
    txt = f"Размерность матрицы H: {shape(H)}"
    report += txt + "\n"
    print(txt)

    # Генерация случайных параметров движения
    s_r = lambda: np.random.uniform(-100, 100)
    s_v = lambda: np.random.uniform(-10, 10)
    s_w = lambda: np.random.uniform(-1e-4, 1e-4)
    q = np.array([s_v() for _ in range(4)])
    q /= np.linalg.norm(q)
    rand_params = [(ω, ω_orb), (pi, np.pi), 
                   (r_cube[0], s_r()), (r_cube[1], s_r()), (r_cube[2], s_r()), 
                   (v_cube[0], s_v()), (v_cube[1], s_v()), (v_cube[2], s_v())]
    for i in range(n):
        tmp_r = [(x[i], s_r()), (y[i], s_r()), (z[i], s_r())]
        tmp_v = [(vx[i], s_v()), (vy[i], s_v()), (vz[i], s_v())]
        rand_params += tmp_r + tmp_v + [(wx[i], s_w()), (wy[i], s_w()), (wz[i], s_w()), 
                                        (q0[i], q[0]), (qx[i], q[1]), (qy[i], q[2]), (qz[i], q[3])]

    # Якобиан матрицы наблюдаемости: J[измерение (H), состояние (X)]
    J = Matrix([[Y[0] for _ in range(len(X))] for _ in range(k * l)])
    J_numb = np.array([[0. for _ in range(len(X))] for _ in range(k * l)])
    for i in range(k * l):
        if testprint:
            print(f"_J_numb: расчёт строки_: {i+1} / {k * l}")
        for j in range(len(X)):
            tmp = H[i].diff(X[j])
            J[i, j] = tmp
            J_numb[i][j] = float(tmp.subs(rand_params))
    txt = f"Размерность матрицы J: {shape(J)}"
    report += txt + "\n"
    print(txt)
    
    # Достаточное условие
    def matrix_minor(arr: np.ndarray, i: int, j: int) -> float:
        return np.linalg.det(np.delete(np.delete(arr,i,axis=0), j, axis=1))
    txt = f"\nРанг матрицы: {[np.linalg.matrix_rank(J_numb, tol=tol) for tol in [1e-3, 1e-5, 1e-7, 1e-10, 1e-12, 1e-15]]}\n"
    if hand_written_deriv is None:
        txt += f"Следующие параметры не должны быть нулевыми:\n"
        report += txt + "\n"
        print(txt)
        d = []
        Δ = []
        flag = True
        i_min = -1
        for i in range(len(X)):
            tmp = matrix_minor(J_numb, i, i)
            d += [tmp if i == 0 else tmp / Δ[-1]]
            Δ += [tmp]
            txt = f"Δ_{i} = {Δ[-1]}"
            report += txt + "\n"
            print(txt)
        
            # Чек наблюдаемости
            if flag:
                if abs(d[-1]) < 1e-10:
                    i_min = i
                    flag = False
            if not flag:
                break
    
        # Вывод
        if flag:
            txt = f"\n\033[1mВыполнено достаточное условие! Система наблюдаема\033[0m"
        else:
            in_txt = f"Δ_{i_min}" if i_min == 0 else f"Δ_{i_min} / Δ_{i_min-1}"
            txt = f"\n\033[1mНе выполнено достаточное условие. Нулевой параметр: {in_txt} = {d[i_min]}\033[0m"
        report += txt + "\n"
        print(txt)
        return H, J, J_numb, Δ, report
    report += txt + "\n"
    print(txt)
    return H, J, J_numb, report

# <span style="color:#00b90e">Применение алгоритма на примерах</span>

#### <span style="color:#2c3e50">Наблюдаемость системы</span> <span style="color:#bb4bca">при кубсате строго на круговой орбите, без аэродинамики и углового движения,</span> <span style="color:#e60b9d">антенны изотропные</span>

In [77]:
# Уравнения движения
def MyDiff_AeroOff_AttitudeOff(expr, power: int = 1):
    """Функция взятия производной по времени.
    Аэродинамика: НЕ УЧИТЫВАЕТСЯ
    Угловое движение: НЕ УЧИТЫВАЕТСЯ
    :param expr: Выражение, от которого надо взять производную по времени t
    :param power: Степень производной"""
    if power == 0:
        return expr
    if power == 1:
        anw = expr
    else:
        anw = my_diff(expr, power - 1)
    subses = []
    for i in range(n):
        subses += [(Derivative(x[i], t), vx[i])]
        subses += [(Derivative(y[i], t), vy[i])]
        subses += [(Derivative(z[i], t), vz[i])]
        subses += [(Derivative(vx[i], t), -2*ω*vz[i])]
        subses += [(Derivative(vy[i], t), -ω**2*y[i])]
        subses += [(Derivative(vz[i], t), 2*ω*vx[i] + 3*ω**2*z[i])]
    return anw.diff(t).subs(subses).simplify()

In [ ]:
# Количество чипсатов
n = 11

# Инициализация переменных
t, ω, _, _, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)

# Состояние в начальный момент времени
X, Y = ([], [])
for i in range(n):
    X += [x[i], y[i], z[i], vx[i], vy[i], vz[i]]

# Измерения в начальный момент времени
for j in range(n):
    for i in range(j+1):
        if i == j:
            Y += [sqrt(x[i]**2 + y[i]**2 + z[i]**2)]
        else:
            Y += [sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2)]

H_1, J_1, J_numb_1, d_1, report_1_11 = ShauyingObservabilitySufficientCondition(testprint=False, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOff_AttitudeOff)

In [29]:
# save_reports([report_1_1, report_1_5, report_1_11], "AeroOff_AttitudeOff_AntennaOff")
print(read_reports("AeroOff_AttitudeOff_AntennaOff"))

Количество чипсатов: 1
Неизвестные: n = 6 (на каждый чипсат по 6 параметров)
Известные: l = 1
∃ производные порядка k = 6.0 (Должна быть целой!)
Размерность матрицы H: (6, 1)
Размерность матрицы J: (6, 6)

Ранг матрицы: [3, 3, 5, 6, 6, 6]
Следующие параметры не должны быть нулевыми:

Δ_0 = 2.4129801856449626e-16

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = 2.4129801856449626e-16

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Количество чипсатов: 5
Неизвестные: n = 30 (на каждый чипсат по 6 параметров)
Известные: l = 15
∃ производные порядка k = 2.0 (Должна быть целой!)
Размерность матрицы H: (30, 1)
Размерность матрицы J: (30, 30)

Ранг матрицы: [24, 24, 24, 24, 24, 24]
Следующие параметры не должны быть нулевыми:

Δ_0 = 5.246074945807661e-86

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = 5.246074945807661e

<span style="color:#0ab49a">Согласно статье</span> <span style="color:#A254FC">055 (Yujiro Inowe) On the Observability of Autonomous Nonlinear Systems</span> <span style="color:#0ab49a">надо просто проверить ранг во всём $R^n$!</span>    $rg\frac{\partial \boldsymbol{H}_d}{\partial \boldsymbol{x}} (\boldsymbol{x}) = n \hskip20px \forall x \in R^n,$
$$d = n \frac{n +3}{2}.$$

In [88]:
# Количество чипсатов
n = 11

# Инициализация переменных
t, ω, _, _, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)

# Состояние в начальный момент времени
X, Y = ([], [])
for i in range(n):
    X += [x[i], y[i], z[i], vx[i], vy[i], vz[i]]

# Измерения в начальный момент времени
for j in range(n):
    for i in range(j+1):
        if i == j:
            Y += [sqrt(x[i]**2 + y[i]**2 + z[i]**2)]
        else:
            Y += [sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2)]

H_1, J_1, J_numb_1, report_1_11 = ShauyingObservabilitySufficientCondition(testprint=True, hand_written_deriv=3, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOff_AttitudeOff)

Количество чипсатов: 11

Внимание! Рассчитывается не отношение миноров, а ранг расширенного Якобиана
Неизвестные: n = 66 (на каждый чипсат по 6 параметров)
Известные: l = 66
∃ производные порядка k = 1.0 (Должна быть целой!)
_расчёт матрицы H_: k=1/3, l=1/66
_расчёт матрицы H_: k=1/3, l=2/66
_расчёт матрицы H_: k=1/3, l=3/66
_расчёт матрицы H_: k=1/3, l=4/66
_расчёт матрицы H_: k=1/3, l=5/66
_расчёт матрицы H_: k=1/3, l=6/66
_расчёт матрицы H_: k=1/3, l=7/66
_расчёт матрицы H_: k=1/3, l=8/66
_расчёт матрицы H_: k=1/3, l=9/66
_расчёт матрицы H_: k=1/3, l=10/66
_расчёт матрицы H_: k=1/3, l=11/66
_расчёт матрицы H_: k=1/3, l=12/66
_расчёт матрицы H_: k=1/3, l=13/66
_расчёт матрицы H_: k=1/3, l=14/66
_расчёт матрицы H_: k=1/3, l=15/66
_расчёт матрицы H_: k=1/3, l=16/66
_расчёт матрицы H_: k=1/3, l=17/66
_расчёт матрицы H_: k=1/3, l=18/66
_расчёт матрицы H_: k=1/3, l=19/66
_расчёт матрицы H_: k=1/3, l=20/66
_расчёт матрицы H_: k=1/3, l=21/66
_расчёт матрицы H_: k=1/3, l=22/66
_расчёт матриц

KeyboardInterrupt: 

----

#### <span style="color:#2c3e50">Наблюдаемость системы</span> <span style="color:#00b0b9">при кубсате строго на круговой орбите, с аэродинамикой,</span> <span style="color:#bb4bca">без углового движения,</span> <span style="color:#e60b9d">антенны изотропные</span>

In [79]:
C = 1.17
S = 0.1 ** 2
m = 0.01  # 10 грамм

# Уравнения движения
def MyDiff_AeroOn_AttitudeOff(expr, power: int = 1):
    """Функция взятия производной по времени.
    Аэродинамика: учитывается
    Угловое движение: НЕ УЧИТЫВАЕТСЯ
    :param expr: Выражение, от которого надо взять производную по времени t
    :param power: Степень производной"""
    if power == 0:
        return expr
    if power == 1:
        anw = expr
    else:
        anw = my_diff(expr, power - 1)
    subses = []
    for i in range(n):
        subses += [(Derivative(x[i], t), vx[i])]
        subses += [(Derivative(y[i], t), vy[i])]
        subses += [(Derivative(z[i], t), vz[i])]
        subses += [(Derivative(vx[i], t), -2*ω*vz[i] - S*C*ρ/m * (v_orb + vy[i])**2)]  # Нет учёта аэродинамики
        subses += [(Derivative(vy[i], t), -ω**2*y[i])]
        subses += [(Derivative(vz[i], t), 2*ω*vx[i] + 3*ω**2*z[i])]
    return anw.diff(t).subs(subses).simplify()

In [85]:
# Количество чипсатов
# {1->6, 5->2, 11->1}
# {1->4, 3->2, 7->1} без у
n = 5

# Инициализация переменных
t, ω, _, _, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)

# Состояние в начальный момент времени
X, Y = ([], [])
for i in range(n):
    X += [x[i], y[i], z[i], vx[i], vy[i], vz[i]]
    # X += [x[i], z[i], vx[i], vz[i]] 

# Измерения в начальный момент времени
for j in range(n):
    for i in range(j+1):
        if i == j:
            Y += [sqrt(x[i]**2 + y[i]**2 + z[i]**2)]
        else:
            Y += [sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2)]
            
# H_11, J_11, J_numb_11, d_11, report_11_ = ShauyingObservabilitySufficientCondition(testprint=False, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOn_AttitudeOff)
H_11, J_11, J_numb_11, report_11_ = ShauyingObservabilitySufficientCondition(testprint=True, hand_written_deriv=4, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOff_AttitudeOff)
# report_11_1 = f"\033[1mВнимание! Нет учёта компоненты y!\033[0m\n" + report_11_

Количество чипсатов: 5

Внимание! Рассчитывается не отношение миноров, а ранг расширенного Якобиана
Неизвестные: n = 30 (на каждый чипсат по 6 параметров)
Известные: l = 15
∃ производные порядка k = 2.0 (Должна быть целой!)
_расчёт матрицы H_: k=1/4, l=1/15
_расчёт матрицы H_: k=1/4, l=2/15
_расчёт матрицы H_: k=1/4, l=3/15
_расчёт матрицы H_: k=1/4, l=4/15
_расчёт матрицы H_: k=1/4, l=5/15
_расчёт матрицы H_: k=1/4, l=6/15
_расчёт матрицы H_: k=1/4, l=7/15
_расчёт матрицы H_: k=1/4, l=8/15
_расчёт матрицы H_: k=1/4, l=9/15
_расчёт матрицы H_: k=1/4, l=10/15
_расчёт матрицы H_: k=1/4, l=11/15
_расчёт матрицы H_: k=1/4, l=12/15
_расчёт матрицы H_: k=1/4, l=13/15
_расчёт матрицы H_: k=1/4, l=14/15
_расчёт матрицы H_: k=1/4, l=15/15
_расчёт матрицы H_: k=2/4, l=1/15
_расчёт матрицы H_: k=2/4, l=2/15
_расчёт матрицы H_: k=2/4, l=3/15
_расчёт матрицы H_: k=2/4, l=4/15
_расчёт матрицы H_: k=2/4, l=5/15
_расчёт матрицы H_: k=2/4, l=6/15
_расчёт матрицы H_: k=2/4, l=7/15
_расчёт матрицы H_: k=

In [37]:
# save_reports([report_11_1, report_11_3, report_11_7], "AeroOn_AttitudeOff_AntennaOff")
print(read_reports("AeroOn_AttitudeOff_AntennaOff"))

Внимание! Нет учёта компоненты y!
Количество чипсатов: 1
Неизвестные: n = 4 (на каждый чипсат по 4 параметров)
Известные: l = 1
∃ производные порядка k = 4.0 (Должна быть целой!)
Размерность матрицы H: (4, 1)
Размерность матрицы J: (4, 4)

Ранг матрицы: [3, 4, 4, 4, 4, 4]
Следующие параметры не должны быть нулевыми:

Δ_0 = 6.568626533566369e-05
Δ_1 = 1.0343177642697167e-05
Δ_2 = 0.0002295493843337985
Δ_3 = 0.004902710294511974

Выполнено достаточное условие! Система наблюдаема

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Внимание! Нет учёта компоненты y!
Количество чипсатов: 3
Неизвестные: n = 12 (на каждый чипсат по 4 параметров)
Известные: l = 6
∃ производные порядка k = 2.0 (Должна быть целой!)
Размерность матрицы H: (12, 1)
Размерность матрицы J: (12, 12)

Ранг матрицы: [10, 10, 10, 10, 10, 10]
Следующие параметры не должны бы

----

#### <span style="color:#2c3e50">Наблюдаемость системы</span> <span style="color:#00b0b9">при кубсате</span> <span style="color:#ed9a00">не на круговой орбите,</span> <span style="color:#00b0b9">с аэродинамикой,</span> <span style="color:#bb4bca">без углового движения,</span> <span style="color:#e60b9d">антенны изотропные</span>

In [72]:
C = 1.17
S = 0.1 ** 2
m = 0.01  # 10 грамм

# Уравнения движения
def MyDiff_AeroOn_AttitudeOff_WithCubeSat(expr, power: int = 1):
    """Функция взятия производной по времени.
    Аэродинамика: учитывается
    Угловое движение: НЕ УЧИТЫВАЕТСЯ
    :param expr: Выражение, от которого надо взять производную по времени t
    :param power: Степень производной"""
    if power == 0:
        return expr
    if power == 1:
        anw = expr
    else:
        anw = my_diff(expr, power - 1)
    subses = [(Derivative(r_cube[0], t), v_cube[0]), 
              (Derivative(r_cube[1], t), v_cube[1]), 
              (Derivative(r_cube[2], t), v_cube[2]),
              (Derivative(v_cube[0], t), -2*ω*v_cube[2] - (0.3)**2*C*ρ/10 * (v_orb + v_cube[1])**2),
              (Derivative(v_cube[1], t), -ω**2*r_cube[1]),
              (Derivative(v_cube[2], t), 2*ω*v_cube[0] + 3*ω**2*r_cube[2])]
    for i in range(n):
        subses += [(Derivative(x[i], t), vx[i])]
        subses += [(Derivative(y[i], t), vy[i])]
        subses += [(Derivative(z[i], t), vz[i])]
        subses += [(Derivative(vx[i], t), -2*ω*vz[i] - S*C*ρ/m * (v_orb + vy[i])**2)]
        subses += [(Derivative(vy[i], t), -ω**2*y[i])]
        subses += [(Derivative(vz[i], t), 2*ω*vx[i] + 3*ω**2*z[i])]
    return anw.diff(t).subs(subses).simplify()

In [ ]:
# Количество чипсатов
# {1->6, 5->2, 11->1}
# {1->4, 3->2, 7->1} без у
n = 11

# Инициализация переменных
t, ω, r_cube, v_cube, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)

# Состояние в начальный момент времени
X, Y = ([], [])
for i in range(n):
    X += [x[i], z[i], y[i], vx[i], vy[i], vz[i]]
    # X += [x[i], z[i], vx[i], vz[i]]

# Измерения в начальный момент времени
for j in range(n):
    for i in range(j+1):
        if i == j:
            Y += [sqrt((x[i]-r_cube[0])**2 + (y[i]-r_cube[1])**2 + (z[i]-r_cube[2])**2)]
            # Y += [sqrt(x[i]**2 + y[i]**2 + z[i]**2)]
        else:
            Y += [sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2)]
            
# H_12, J_12, J_numb_12, d_12, report_12_11 = ShauyingObservabilitySufficientCondition(testprint=True, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOn_AttitudeOff_WithCubeSat)
H_12, J_12, J_numb_12, report_12_11 = ShauyingObservabilitySufficientCondition(testprint=True, hand_written_deriv=3, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOn_AttitudeOff_WithCubeSat)
# report_12_11 = f"\033[1mВнимание! Нет учёта компоненты y!\033[0m\n" + report_12_

In [87]:
print(report_12_11)

Количество чипсатов: 11
Неизвестные: n = 66 (на каждый чипсат по 6 параметров)
Известные: l = 66
∃ производные порядка k = 1.0 (Должна быть целой!)
Размерность матрицы H: (198, 1)
Размерность матрицы J: (198, 66)

Ранг матрицы: [63, 65, 66, 66, 66, 66]




----

#### <span style="color:#2c3e50">Наблюдаемость системы</span> <span style="color:#00b0b9">при кубсате строго на круговой орбите, с аэродинамикой и угловым движением,</span> <span style="color:#e60b9d">антенны изотропные</span>

In [5]:
# Уравнения движения
def MyDiff_AeroOn_AttitudeOn(expr, power: int = 1):
    """Функция взятия производной по времени.
    Аэродинамика: учитывается
    Угловое движение: учитывается
    Примечание: крутящий момент равен 0 (даже гравитационного нет)
    :param expr: Выражение, от которого надо взять производную по времени t
    :param power: Степень производной"""
    if power == 0:
        return expr
    if power == 1:
        anw = expr
    else:
        anw = my_diff(expr, power - 1)
    subses = []
    for i in range(n):
        M = np.zeros(3)
        subses += [(Derivative(x[i], t), vx[i])]
        subses += [(Derivative(y[i], t), vy[i])]
        subses += [(Derivative(z[i], t), vz[i])]
        subses += [(Derivative(vx[i], t), -2*ω*vz[i] - S*C*ρ/m * (v_orb + vy[i])**2 * (qx[i]*qy[i] - q0[i]*qz[i]))]
        subses += [(Derivative(vy[i], t), -ω**2*y[i])]
        subses += [(Derivative(vz[i], t), 2*ω*vx[i] + 3*ω**2*z[i])]
        subses += [(Derivative(q0[i], t), (-wx[i]*qx[i] - wy[i]*qy[i] - wz[i]*qz[i])/2)]
        subses += [(Derivative(qx[i], t), (wx[i]*q0[i] + wy[i]*qz[i] - wz[i]*qy[i])/2)]
        subses += [(Derivative(qy[i], t), (wy[i]*q0[i] + wz[i]*qx[i] - wx[i]*qz[i])/2)]
        subses += [(Derivative(qz[i], t), (wz[i]*q0[i] + wx[i]*qy[i] - wy[i]*qx[i])/2)]
        subses += [(Derivative(wx[i], t), (Jyy*wy[i]*wz[i] - Jzz*wy[i]*wz[i] + M[0]) / Jxx)]
        subses += [(Derivative(wy[i], t), (-Jxx*wx[i]*wz[i] + Jzz*wx[i]*wz[i] + M[1]) / Jyy)]
        subses += [(Derivative(wz[i], t), (Jxx*wx[i]*wy[i] - Jyy*wx[i]*wy[i] + M[2]) / Jzz)]
    return anw.diff(t).subs(subses).simplify()

In [ ]:
# Количество чипсатов
# {1->12, 3->6, 5->4, 7->3, 11->2, 23->1}
n = 11

# Коэффициент сопротивления
C = 1.17
S = 0.1 ** 2
m = 0.01  # 10 грамм
Jxx, Jyy, Jzz = (0.01, 0.01, 0.007)

# Инициализация переменных
t, ω, _, _, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)

# Состояние в начальный момент времени
X, Y = ([], [])
for i in range(n):
    X += [x[i], y[i], z[i], vx[i], vy[i], vz[i], qx[i], qy[i], qz[i], wx[i], wy[i], wz[i]]  # , q0[i]

# Измерения в начальный момент времени
for j in range(n):
    for i in range(j+1):
        if i == j:
            Y += [sqrt(x[i]**2 + y[i]**2 + z[i]**2)]
        else:
            Y += [sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2)]

H_2, J_2, J_numb_2, d_2, report_2_ = ShauyingObservabilitySufficientCondition(testprint=True, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOn_AttitudeOn)

In [37]:
# save_reports([report_2_], "AeroOn_AttitudeOn_AntennaOff")
print(read_reports("AeroOn_AttitudeOn_AntennaOff"))

Количество чипсатов: 23
Неизвестные: n = 276 (на каждый чипсат по 12 параметров)
Известные: l = 276
∃ производные порядка k = 1.0 (Должна быть целой!)
Размерность матрицы H: (276, 1)
Размерность матрицы J: (276, 276)

Ранг матрицы: 66
Следующие параметры не должны быть нулевыми:

Δ_0 = 0.0

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = 0.0



----

#### <span style="color:#2c3e50">Наблюдаемость системы</span> <span style="color:#00b0b9">при кубсате строго на круговой орбите,</span> <span style="color:#00b0b9">с аэродинамикой и угловым движением,</span> <span style="color:#15cbfc">антенны анизотропны</span>

In [6]:
# Количество чипсатов
# {11->1}
n = 11

# Разложение по антеннам
antennas = "xy"
print(f"\033[1mУ каждого чипсата {len(antennas)} антенн(ы): {antennas}\033[0m")

# Коэффициент сопротивления
C = 1.17
m = 0.01  # 10 грамм
Jxx, Jyy, Jzz = (0.01, 0.01, 0.007)

# Инициализация переменных
t, ω, _, _, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)

# Состояние в начальный момент времени
X, r, Y = ([], [], [])
for i in range(n):  
    X += [x[i], y[i], z[i], vx[i], vy[i], vz[i], qx[i], qy[i], qz[i], wx[i], wy[i], wz[i]]  # , q0[i]

# Измерения в начальный момент времени
for j in range(n):
    for i in range(j+1):
        if i == j:
            tmp = sqrt(x[i]**2 + y[i]**2 + z[i]**2)
            Y += [tmp / sqrt(local_dipol([x[i], y[i], z[i]], tmp, c, [q0[i], qx[i], qy[i], qz[i]])) for c in antennas]
        else:
            tmp = sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2)
            Y += [tmp / sqrt(local_dipol([x[i]-x[j], y[i]-y[j], z[i]-z[j]], tmp, c, [q0[i], qx[i], qy[i], qz[i]])) \
                      / sqrt(local_dipol([x[j]-x[i], y[j]-y[i], z[j]-z[i]], tmp, c, [q0[j], qx[j], qy[j], qz[j]])) for c in antennas]

H_3, J_3, J_numb_3, d_3, report_3_ = ShauyingObservabilitySufficientCondition(testprint=True, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOn_AttitudeOn)
report_3_ = f"\033[1mУ каждого чипсата {len(antennas)} антенн(ы): {antennas}\033[0m\n" + report_3_

У каждого чипсата 2 антенн(ы): xy
Количество чипсатов: 11

Неизвестные: n = 132 (на каждый чипсат по 12 параметров)
Известные: l = 132
∃ производные порядка k = 1.0 (Должна быть целой!)
_расчёт матрицы H_: k=1/1, l=1/132
_расчёт матрицы H_: k=1/1, l=2/132
_расчёт матрицы H_: k=1/1, l=3/132
_расчёт матрицы H_: k=1/1, l=4/132
_расчёт матрицы H_: k=1/1, l=5/132
_расчёт матрицы H_: k=1/1, l=6/132
_расчёт матрицы H_: k=1/1, l=7/132
_расчёт матрицы H_: k=1/1, l=8/132
_расчёт матрицы H_: k=1/1, l=9/132
_расчёт матрицы H_: k=1/1, l=10/132
_расчёт матрицы H_: k=1/1, l=11/132
_расчёт матрицы H_: k=1/1, l=12/132
_расчёт матрицы H_: k=1/1, l=13/132
_расчёт матрицы H_: k=1/1, l=14/132
_расчёт матрицы H_: k=1/1, l=15/132
_расчёт матрицы H_: k=1/1, l=16/132
_расчёт матрицы H_: k=1/1, l=17/132
_расчёт матрицы H_: k=1/1, l=18/132
_расчёт матрицы H_: k=1/1, l=19/132
_расчёт матрицы H_: k=1/1, l=20/132
_расчёт матрицы H_: k=1/1, l=21/132
_расчёт матрицы H_: k=1/1, l=22/132
_расчёт матрицы H_: k=1/1, l=23


KeyboardInterrupt



In [9]:
# save_reports([report_3_13], "AeroOn_AttitudeOn_AntennaOn")
print(read_reports("AeroOn_AttitudeOn_AntennaOn"))

У каждого чипсата 2 антенн(ы): xy
Количество чипсатов: 11
Неизвестные: n = 132 (на каждый чипсат по 12 параметров)
Известные: l = 132
∃ производные порядка k = 1.0 (Должна быть целой!)
Размерность матрицы H: (132, 1)
Размерность матрицы J: (132, 132)

Ранг матрицы: 82
Следующие параметры не должны быть нулевыми:

Δ_0 = 0.0

Не выполнено достаточное условие. Нулевой параметр: Δ_0 = 0.0



----

#### <span style="color:#2c3e50">То же самое, но</span> <span style="color:#00b0b9">диаграмма направленностей не симметрична</span>

In [55]:
# Количество чипсатов
n = 11

# Разложение по антеннам
antennas = "xy"
print(f"\033[1mУ каждого чипсата {len(antennas)} антенн(ы): {antennas}\033[0m")

# Коэффициент сопротивления
C = 1.17
m = 0.01  # 10 грамм
Jxx, Jyy, Jzz = (0.01, 0.01, 0.007)
distortion = 0.4

# Инициализация переменных
t, ω, _, _, x, y, z, vx, vy, vz, wx, wy, wz, q0, qx, qy, qz = get_params(n)

# Состояние в начальный момент времени
X, r, Y = ([], [], [])
for i in range(n):
    X += [x[i], y[i], z[i], vx[i], vy[i], vz[i], qx[i], qy[i], qz[i], wx[i], wy[i], wz[i]]  # , q0[i]

# Измерения в начальный момент времени
for j in range(n):
    for i in range(j+1):
        if i == j:
            tmp = sqrt(x[i]**2 + y[i]**2 + z[i]**2)
            Y += [tmp / sqrt(local_dipol([x[i], y[i], z[i]], tmp, c, [q0[i], qx[i], qy[i], qz[i]], distortion=distortion)) for c in antennas]
        else:
            tmp = sqrt((x[i]-x[j])**2 + (y[i]-y[j])**2 + (z[i]-z[j])**2)
            Y += [tmp / sqrt(local_dipol([x[i]-x[j], y[i]-y[j], z[i]-z[j]], tmp, c, [q0[i], qx[i], qy[i], qz[i]], distortion=distortion)) \
                      / sqrt(local_dipol([x[j]-x[i], y[j]-y[i], z[j]-z[i]], tmp, c, [q0[j], qx[j], qy[j], qz[j]], distortion=distortion)) for c in antennas]

H_4, J_4, J_numb_4, d_4, report_4_ = ShauyingObservabilitySufficientCondition(testprint=True, n=n, X=X, Y=Y, my_diff=MyDiff_AeroOn_AttitudeOn)
report_4_ = f"\033[1mУ каждого чипсата {len(antennas)} кривые антенн(ы): {antennas}\033[0m\n" + report_4_

У каждого чипсата 2 антенн(ы): xy
Количество чипсатов: 11

Неизвестные: n = 132 (на каждый чипсат по 12 параметров)
Известные: l = 132
∃ производные порядка k = 1.0 (Должна быть целой!)
_расчёт матрицы H_: k=1/1, l=1/132
_расчёт матрицы H_: k=1/1, l=2/132
_расчёт матрицы H_: k=1/1, l=3/132
_расчёт матрицы H_: k=1/1, l=4/132
_расчёт матрицы H_: k=1/1, l=5/132
_расчёт матрицы H_: k=1/1, l=6/132
_расчёт матрицы H_: k=1/1, l=7/132
_расчёт матрицы H_: k=1/1, l=8/132
_расчёт матрицы H_: k=1/1, l=9/132
_расчёт матрицы H_: k=1/1, l=10/132
_расчёт матрицы H_: k=1/1, l=11/132
_расчёт матрицы H_: k=1/1, l=12/132
_расчёт матрицы H_: k=1/1, l=13/132
_расчёт матрицы H_: k=1/1, l=14/132
_расчёт матрицы H_: k=1/1, l=15/132
_расчёт матрицы H_: k=1/1, l=16/132
_расчёт матрицы H_: k=1/1, l=17/132
_расчёт матрицы H_: k=1/1, l=18/132
_расчёт матрицы H_: k=1/1, l=19/132
_расчёт матрицы H_: k=1/1, l=20/132
_расчёт матрицы H_: k=1/1, l=21/132
_расчёт матрицы H_: k=1/1, l=22/132
_расчёт матрицы H_: k=1/1, l=23

/tmp/ipykernel_4415/3097828645.py:74: RuntimeWarning: invalid value encountered in scalar divide
  d += [tmp if i == 0 else tmp / d[-1]]


In [58]:
J_numb_4[0][1]

0.4555037547125389

In [ ]:
# save_reports([report_4_9xyz, report_4_13xy], "AeroOn_AttitudeOn_AntennaOn_CurveRadiation")
print(read_reports("AeroOn_AttitudeOn_AntennaOn_CurveRadiation"))

### <span style="color:#0ab49a">Что-то с производной</span> <span style="color:#A254FC">Ли</span>

In [ ]:
for i in range(n):
    X_ += [x[i], y[i], z[i], vx[i], vy[i], vz[i]]
def GetF():
    anw = []
    for i in range(n):
        anw += [vx[i], vy[i], vz[i]]
        anw += [2*ω*vz[i], -ω**2*y[i], 2*ω*vx[i] + 3*ω**2*z[i]]
    return Matirx(anw)

# Уравнения движения
def GetGradient_AeroOff_AttitudeOff_AntennaOff(expr):
    """Функция взятия производной по времени.
    Аэродинамика: НЕ УЧИТЫВАЕТСЯ
    Угловое движение: НЕ УЧИТЫВАЕТСЯ
    :param expr: Выражение, от которого надо взять производную по времени t
    :param power: Степень производной"""
    subses = []
    for i in range(n):
        subses += [(Derivative(x[i], t), vx[i])]
        subses += [(Derivative(y[i], t), vy[i])]
        subses += [(Derivative(z[i], t), vz[i])]
        subses += [(Derivative(vx[i], t), -2*ω*vz[i])]  # Нет учёта аэродинамики
        subses += [(Derivative(vy[i], t), -ω**2*y[i])]
        subses += [(Derivative(vz[i], t), 2*ω*vx[i] + 3*ω**2*z[i])]
    return expr.diff(vari).subs(subses).simplify()